## Exploratory data analysis
**Изучение данных**

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 70)   # Настройка отображения данных в Jupyter notebook
pd.set_option('display.max_rows', 100)
pd.set_option('precision', 3)

%pylab inline
# Visualization and Graphics
%matplotlib inline
import matplotlib.pyplot as plt

# !conda install seaborn 
import seaborn as sns
plt.rcParams['figure.figsize'] = (7,7)   # (8,6)

#!pip install ggplot
matplotlib.style.use('ggplot')       # Use ggplot style plots

In [ ]:
# Чтение данных из сохранённого файла

df_ext = pd.DataFrame()
df_ext = pd.read_csv('./Wilson_ext.csv', sep=';', encoding='utf-8') 
print(df_ext.columns)

### Группировка и комбинирование признаков
#### Группы признаков
Группы выделены в соответствии со смыслом данных, логикой их использования и обработки, форматом представления данных.
<br> Группы сформированы в файле `preprocess_data__New.ipynb`
- `target`
- `relatives`
- `sex`, `sex_cat`
- `bmi`, `bmi_scaled`
- `symptom`
- `cirrhosis`
- `debut_age`, `debut_age_scaled`
- `debut_organ`
- `genetic`
- `genetic_dummy`
- `genetic__1`, `genetic__2`
- `genetic_risk__1`, `genetic_risk__2`
- `genetic_risk__1_scaled`, `genetic_risk__2_scaled`
- `data`
- `exclude`, `exclude_model`

<br> Вспомогательные признаки
- `num_to_scale`
- `num_scaled`
- `genetic_risk`
- `genetic_risk_scaled`

In [ ]:
# !conda install simplejson
%run data_to_json

In [ ]:
from collections import OrderedDict

In [ ]:
features_file = open('features_file.json', 'r')
features = json_to_data( features_file.read() )
features_file.close()

print(features)
type(features)

#### Комбинация признаков

In [ ]:
def combine_features(all_feat_list, exclude_features):
    combined_list = list()
    for sublist in all_feat_list:
        for item in sublist:
            combined_list.append(item)
    combined_list = [x for x in combined_list if x not in exclude_features]
    return(combined_list)

### Первичное описание данных

In [ ]:
df_ext.describe().T

### Корреляция признаков с целевым показателем: `TargetHead`

In [ ]:
import scipy.stats  as stats

correlation = list()
for i in range(len(df_ext.columns)):
    col_x = df_ext.columns[i]
    #print(col_x)
    df_xy = df_ext[[col_x, 'TargetHead']].dropna()
    x, y = df_xy.iloc[:,0].values, df_xy.iloc[:,1].values
    try:
        x = [float(i) for i in x]
        y = [float(i) for i in y]
        
        corr_pearson = stats.pearsonr(x, y)
        corr_pearson_value   = corr_pearson[0]
        corr_pearson_p_value = corr_pearson[1]
        #corr_s = stats.spearmanr(x, y)
        if corr_pearson_p_value < 0.1:
            correlation.append([col_x, corr_pearson_value, corr_pearson_p_value])    #, corr_s ])
        #print(col_x, np.unique(x))
    except:
        pass

correlation

In [ ]:
df_ext.corr()['TargetHead']

### Анализ Cirrhosis features
Что в данных:
* Признак 'Cirrhosis' слабее 'Advanced'. В одном случае - неоднозначность сравнения.
* Признаки 'Cirrhosis' слабее 'ChildPugh'. В одном случае - неоднозначность сравнения.
* Признаки 'Advanced' и 'ChildPugh' дополняют друг друга, признак 'ChildPugh' сильнее.

#### По итогам анализа
* Проверить аномалии - нет ли в данных опечатки: <br> (df_ext['Cirrhosis'] == 1) &  (df_ext['Advanced'] == 1) <br> (df_ext['Advanced'] == 1) &  (df_ext['ChildPugh'] == 3) <br> (df_ext['Cirrhosis'] == 1) &  (df_ext['ChildPugh'] == 1)
* Попробовать использовать в модели один признак для описания цирроза, начиная со слабого: \['Cirrhosis', 'Advanced', 'ChildPugh'\]. Прверить, как зависит качество предсказания от силы признака.

In [ ]:
features['cirrhosis']

In [ ]:
# Признаки 'Cirrhosis' и 'ChildPugh'

print( np.unique(df_ext['Advanced']) )
print( np.unique(df_ext['Cirrhosis']) )

print( ( len(df_ext[ (df_ext['Cirrhosis'] == 1) &  (df_ext['Advanced'] == 0) ].index),
         len(df_ext[ (df_ext['Cirrhosis'] == 1) &  (df_ext['Advanced'] == 1) ].index) ) )

plt.figure(1)

plt.subplot(121)
plt.rcParams['figure.figsize'] = (9,2)

plt.hist2d(df_ext['Advanced'], df_ext['Cirrhosis'], bins=[3,2]) #, cmap = my_cmap, norm=LogNorm())
plt.xlabel('Advanced')
plt.ylabel('Cirrhosis')
plt.colorbar()

plt.subplot(122)
plt.scatter( np.array(df_ext['Advanced']), np.array(df_ext['Cirrhosis']) )
plt.xlabel('Advanced')
plt.ylabel('\nCirrhosis')

plt.show()


In [ ]:
# Признаки 'Advanced' и 'ChildPugh'

print( np.unique(df_ext['ChildPugh']) )
print( np.unique(df_ext['Advanced']) )

print( ( len(df_ext[ (df_ext['Advanced'] == 1) &  (df_ext['ChildPugh'] == 2) ].index),
         len(df_ext[ (df_ext['Advanced'] == 2) &  (df_ext['ChildPugh'] == 2) ].index),
         len(df_ext[ (df_ext['Advanced'] == 1) &  (df_ext['ChildPugh'] == 3) ].index),
         len(df_ext[ (df_ext['Advanced'] == 2) &  (df_ext['ChildPugh'] == 3) ].index) ) 
     )

plt.figure(1)

plt.subplot(121)
plt.rcParams['figure.figsize'] = (12,2)
plt.hist2d(df_ext['ChildPugh'], df_ext['Advanced'], bins=[4,3]) #, norm=LogNorm())
plt.xlabel('ChildPugh')
plt.ylabel('Advanced')
plt.colorbar()

plt.subplot(122)
plt.scatter( np.array(df_ext['ChildPugh']), np.array(df_ext['Advanced']) )
plt.xlabel('ChildPugh')
plt.ylabel('Advanced')

plt.show()

In [ ]:
# Признаки 'Cirrhosis' и 'ChildPugh'

print( np.unique(df_ext['ChildPugh']) )
print( np.unique(df_ext['Cirrhosis']) )

print( ( len(df_ext[ (df_ext['Cirrhosis'] == 1) &  (df_ext['ChildPugh'] == 0) ].index),
         len(df_ext[ (df_ext['Cirrhosis'] == 1) &  (df_ext['ChildPugh'] == 1) ].index) ) )

plt.figure(1)

plt.subplot(121)
plt.rcParams['figure.figsize'] = (12,2)
plt.hist2d(df_ext['ChildPugh'], df_ext['Cirrhosis'], bins=[4,2]) #, norm=LogNorm())
plt.xlabel('ChildPugh')
plt.ylabel('Cirrhosis')
plt.colorbar()

plt.subplot(122)
plt.scatter( np.array(df_ext['ChildPugh']), np.array(df_ext['Cirrhosis']) )
plt.xlabel('ChildPugh')
plt.ylabel('Cirrhosis')

plt.show()
